In [ ]:
# THIS IS THE FILE THAT WE WILL USE TO CREATE THE CODE THAT WILL BE UPDATED ON TO THE CPP APP

In [52]:
from openpyxl import Workbook
from datetime import datetime 
from openpyxl.styles import PatternFill, Font, Alignment, NamedStyle, Border, Side
import os
import pandas as pd

COLOR_PALETTE = {
    "maroon": "C00000",

    #Banner Colors
    "lavender":"D5B8EA",
    "pink": "FF8FFF",
    "blue": "BDD7F1",
    "peach": "F9D2BD",
    "green": "D6EDBD",
    "cyan": "D9FFFF",
    
    }

def apply_color(cell, color_name):
    """Applies a predefined color to a cell."""
    color_code = COLOR_PALETTE.get(color_name, "FFFFFF")  # Default to white if not found
    fill = PatternFill(start_color=color_code, end_color=color_code, fill_type="solid")
    cell.fill = fill


def get_banner_color(curr_year):
    """This functions inputs the current year and returns the color required for the banner

    Args:
        curr_year (String): current year

    Returns:
        String: respective color for year
    """
    end=int(curr_year[-1])
    if end == 4 or end == 9:
        return "green"

    elif end == 5 or end == 0:
        return "pink"

    elif end == 6  or end == 1:
        return "blue"

    elif end == 7  or end == 2 :
        return "peach"

    elif end == 8 or end == 3 :
        return "lavender"

def create_salaries_tab(wb, params_df):

    #GLOBAL VARIABLES 
    year2_string = params_df.loc[params_df["Parameters"] == "year 2 string", "Values"].iloc[0]
    ws = wb.active
    ws.title = f"{year2_string} SALARIES"
    ws.sheet_properties.tabColor = "C00000"

    # FORMATING STYLES

    percentage_style = NamedStyle(name="percentage")
    percentage_style.number_format = '0.00%'

    date_style = NamedStyle(name="date_style")
    date_style.number_format = 'yyyy-mm-dd;@'

    # accounting formating
    accounting_style = '_("$"* #,##0.00_);_("$"* \\(#,##0.00\\);_("$"* "-"??_);_(@_)'

    general_style = NamedStyle(name="general")
    general_style.number_format = 'General'

    # COLUMN DIMENSION PRESETS

    column_dimensions = [
        ("A", 13.86), ("B", 21.86), ("C", 11.43), ("D", 11.43), ("E", 16.14), 
        ("F", 19), ("G", 16.14), ("H", 9.86), ("I", 14.14), ("J", 22.86), 
        ("K", 14.29), ("L", 14.29), ("M", 14.29), ("N", 2), ("O", 14.29), ("P", 2),
        ("Q", 19.20),("R", 5.57),("S", 53.29)
    ]

    for col_name, col_width in column_dimensions:
        ws.column_dimensions[col_name].width = col_width

    # TITLE SETTINGS
    ws.row_dimensions[3].height= 28.50
    title_cell = ws.cell(row=3, column=1)
    title_cell.value = f"{year2_string} SALARY & FRINGE W/COLAS"
    title_cell.font = Font(size=22, bold=True)

    # TIME STAMP SETTINGS
    time_stamp_cell = ws.cell(row=3, column=8)
    apply_color(time_stamp_cell, "maroon")
    time_stamp_cell.value = "Generated on " + datetime.now().strftime("%b-%d-%Y") + " at " + datetime.now().strftime("%H:%M:%S")
    time_stamp_cell.font = Font(size=14, color="FFFFFF", bold=True)  
    ws.merge_cells(start_row=3, start_column=8, end_row=3, end_column=10)

    # FY BANNER SETTINGS
    ws.row_dimensions[1].height = 50.25
    banner =""
    for i in range(1,30):
        if i % 2 == 0:
            banner = banner + "    -    "
        else:
            banner = banner + year2_string

    banner_cell = ws.cell(row=1, column=1)
    banner_cell.value = banner
    banner_cell.font=Font(name = "Ariel", size=18, color= "2F75B5" , bold=  True)
    banner_cell.alignment = Alignment(vertical="center")

    # banner Color
    for col in range(1, 25):
        cell = ws.cell(row=1, column=col)
        apply_color(cell,get_banner_color(year2_string))

    # PRESETS FOR LINES 5-9

    font_blue59 = Font(color="0000CC",bold=  True)

    cell_presets = {
    5: {
        1: {"value": f"{year2_string} Fringe - Academic Staff", "font": font_blue59},                                                               # Column A (1)
        3: {"value": params_df.loc[params_df["Parameters"] == "academic_fringe", "Values"].iloc[0], "style": percentage_style, "fill": "cyan"},     # Column C (3)
        6: {"value": f"{year2_string} Fiscal Year Start", "font": font_blue59},                                                                     # Column F (6)
        7: {"value": params_df.loc[params_df["Parameters"] == "year_start", "Values"].iloc[0], "style": date_style, "fill": "cyan"},                # Column G (7)
    },
    6: {
        1: {"value": f"{year2_string} Fringe - University Staff", "font": font_blue59},                                                             # Column A (1)
        3: {"value": params_df.loc[params_df["Parameters"] == "university_fringe", "Values"].iloc[0], "style": percentage_style, "fill": "cyan"},   # Column C (3)
        6: {"value": f"{year2_string} Fiscal Year End", "font": font_blue59},                                                                       # Column F (6)
        7: {"value": params_df.loc[params_df["Parameters"] == "year_end", "Values"].iloc[0], "style": date_style, "fill": "cyan"},                  # Column G (7)
    },
    7: {
        1: {"value": f"{year2_string} Fringe - Student Staff", "font": font_blue59},                                                                # Column A (1)
        3: {"value": params_df.loc[params_df["Parameters"] == "student_fringe", "Values"].iloc[0], "style": percentage_style, "fill": "cyan"},      # Column C (3)
        6: {"value": f"{year2_string} # Total Days", "font": font_blue59},                                                                          # Column F (6)
        7: {"value": "=(G6-G5)+1", "fill": "cyan"},                                                                                                 # Column G (7)
    },
    9: {
        6: {"value": f"{year2_string} PAY PERIODS", "font": font_blue59},                                                                           # Column F (6)
        7: {"value": params_df.loc[params_df["Parameters"] == "number of pay periods", "Values"].iloc[0], "fill": "cyan"},                          # Column G (7)
    },
    }

    # Iterate over the dictionary to apply settings
    for row, columns in cell_presets.items():
        for col, settings in columns.items():
            cell = ws.cell(row=row, column=col)  # Use numeric column directly
            if "value" in settings:
                cell.value = settings["value"]
            if "font" in settings:
                cell.font = settings["font"]
            if "style" in settings:
                cell.style = settings["style"]
            if "fill" in settings:
                apply_color(cell, settings["fill"])

    

    
    
    return

In [53]:
# This is the code that will go in the MAIN CODE GENERATION SECTION

from openpyxl import Workbook
from datetime import datetime 
import os
import pandas as pd
# HARD CODED VALUES THAT NEED TO BE MADE AVAILABLE

# NOTE: for reference always use the second year to name the all sheet FY's. example, if the 2 years are 2024 and 2025 it would be FY25

def create_payroll_spreadsheet(destination_path, params_df):
    """ This is the entry point for the Payroll Spreadsheet all subfunctions are called from here

    Args:
        destination_path (String): Path of the save destination for the payroll spreadsheet
        params_df (Pandas df): Contains all the paramneters and values for the spreadsheet

    Returns:
        String: Saved location of the Payroll Sheet 
    """
    wb = Workbook()
    wb.active
    create_salaries_tab(wb, params_df)
    create_staff_tab(wb, params_df)
    final_save_path = os.path.join(destination_path,get_save_name(f"{params_df.loc[params_df["Parameters"] == "year 2 string", "Values"].iloc[0]} Payroll_Spreadsheet"))
    wb.save(final_save_path)
    wb.close()
    return final_save_path


def create_staff_tab(wb, params_df):
    ws= wb.create_sheet(title =f"{params_df.loc[params_df["Parameters"] == "year 2 string", "Values"].iloc[0]} STAFF")
    ws.sheet_properties.tabColor = "0000CC"
    return

def get_save_name(spreadsheet_name):
    """ This Function forms the new file name using the nomencalture which is {filename_currentDate_cuurentTime)

    Args:
        spreadsheet_name (String): Spreadsheet Base name

    Returns:
        String: Formatted name of file path to be saved
    """
    current_datetime = datetime.now().strftime("%b-%d-%Y_%H-%M-%S")
    return f"{spreadsheet_name}_{current_datetime}.xlsx"


In [55]:
# THIS IS THE CODE THAT CALLS THE GENERATINO FUNCTION
import pandas as pd
num_pay_periods = 25
y1 = 2024
y2 = 2025

data = {
    "Parameters": ["number of pay periods", "year 1 numeric" , "year 1 string", "year 2 numeric" , "year 2 string","academic_fringe",
                   "university_fringe","student_fringe","year_start","year_end"],
    "Values": [f"{num_pay_periods}" , f"{y1 % 100}", f"FY {y1 % 100}",  f"{y2 % 100}", f"FY {y2 % 100}","N/A Too be filled",
               "N/A Too be filled","N/A Too be filled","N/A Too be filled","N/A Too be filled"]
}
df = pd.DataFrame(data)
temporary_save_path = r"C:\Users\dpaul5\Desktop\2025 APP TESTING"  # Corrected path
os.startfile(create_payroll_spreadsheet(temporary_save_path, df))


df



,Parameters,Values
0,number of pay periods,25
1,year 1 numeric,24
2,year 1 string,FY 24
3,year 2 numeric,25
4,year 2 string,FY 25
5,academic_fringe,N/A Too be filled
6,university_fringe,N/A Too be filled
7,student_fringe,N/A Too be filled
8,year_start,N/A Too be filled
9,year_end,N/A Too be filled


In [153]:
import app_util as apu

# Example Usage
excel_file = r"C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\CPP FY2025 Input Data Sheet 1 - Employees & Funding Strings.xlsx"
tables= apu.extract_table_boundaries(excel_file)
sum =0 
for table in tables:
    df = apu.read_excel_table(excel_file,"USER INPUT",table)
    print(df)
    sum = sum + len(df)
    print(len(df))

print(f"SUM: {sum}")

Processing sheet: USER INPUT
A50:C55
A43:C48
A36:C41
A29:C34
A17:C27
  Employees Paid By Other Departments  List Order         Name
0                                None         1.0  Summer TA 1
1                                None         2.0  Summer TA 2
2                                None         3.0         None
3                                None         4.0         None
4                                None         NaN         None
5
  Graduate Student Employees  List Order          Name
0                       None         1.0   Elisa Lopez
1                       None         2.0        Law PA
2                       None         3.0  Project PA 1
3                       None         4.0  Project PA 2
4                       None         NaN          None
5
  Undergraduate Student Employees  List Order           Name
0                            None         1.0       Dev Paul
1                            None         2.0  Maneeya Leung
2                            None   

In [212]:
# This code generates input sheet 2

import app_util as apu
from openpyxl import Workbook
import re
import openpyxl

#sotring all the headings and its values in a list of tuples
# Nomenclature = {"CONTENT OF HEADING", "ROW NUMBER", "COLUMN NUMBER"}
headings =[
    {"UPDATES",3,1},
    {"FRINGE TABLE",15,1},
    {"TERM LEAVE TABLE",},
    {"CPP EMPLOYEES & SALARY DATA"},
    {"COLAS (Cost of Living Increases)"},
    {"FUNDING STRINGS & ACCOUNTING -- SALARY"},
    {"FUNDING STRINGS & ACCOUNTING -- NON-SALARY"},
]

def generate_input_sheet_2(input_sheet_1_path,destination_path):
    wb = Workbook()
    wb.active
    create_user_input_tab(wb)
    year = re.search(r'FY(\d{4})', input_sheet_1_path).group(1) if re.search(r'FY(\d{4})', input_sheet_1_path) else None

    final_save_path = os.path.join(destination_path,apu.get_save_name(f"CPP FY{year} Input Data Sheet 2 - ALL INPUT DATA"))
    wb.save(final_save_path)
    wb.close()
    return final_save_path

def create_user_input_tab(wb):
    # setting tab title
    ws = wb.active
    ws.title = f"USER INPUT"

    # TITLE SETTINGS
    ws.row_dimensions[1].height= 28.50
    title_cell = ws.cell(row=1, column=1)
    title_cell.value = f"INPUT DATA TABLE 2 - ALL INPUT DATA"
    title_cell.font = Font(size=22, bold=True)

    # TIME STAMP SETTINGS
    time_stamp_cell = ws.cell(row=1, column=10)
    apu.apply_color(time_stamp_cell, "maroon")
    time_stamp_cell.value = "Generated on " + datetime.now().strftime("%b-%d-%Y") + " at " + datetime.now().strftime("%H:%M:%S")
    time_stamp_cell.font = Font(size=14, color="FFFFFF", bold=True)  
    ws.merge_cells(start_row=1, start_column=10, end_row=1, end_column=16) 

    # Set width of the first 200 columns and height of the first 200 rows
    for col in range(1, 201):  # Columns A to GR
        ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width = 25
    for row in range(3, 201):  # Rows 1 to 200
        ws.row_dimensions[row].height = 25

    return



def print_headers(wb):
    
    return  

In [248]:
#TEMPORARILY HARD CODED VALUE FOR TESTING PURPOSES
input = r"C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\CPP FY2025 Input Data Sheet 1 - Employees & Funding Strings.xlsx"
ouput = r"C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\OUTPUTS"
os.startfile(generate_input_sheet_2(input, ouput))

In [167]:
# Initialize an empty list to store DataFrames
dataframes_list = []

# Extract table boundaries from the Excel file
tables = apu.extract_table_boundaries(excel_file)

# Loop through each table, process it, and append to the list
for table in tables:
    # Read the table into a DataFrame
    df = apu.read_excel_table(excel_file, "USER INPUT", table)
    
    # Append the DataFrame to the list
    dataframes_list.append(df)

# Reverse the order of the list
dataframes_list = dataframes_list[::-1]

# Output the reversed list
dataframes_list



[  Salaried Employees  List Order              Name
 0               None         1.0       Sarah Davis
 1               None         2.0       Chris Lacey
 2               None         3.0      Sachin Gupte
 3               None         4.0  Curran Cauldwood
 4               None         5.0         Levi Katz
 5               None         6.0  J. Ashleigh Ross
 6               None         8.0         Sam Russo
 7               None         9.0              None
 8               None        10.0              None
 9               None         NaN              None,
   Lump Sum/No Fringe Employees - Instructors  List Order           Name
 0                                       None         1.0  Jill Jacklitz
 1                                       None         2.0   Hanna Barton
 2                                       None         3.0   Instructor 2
 3                                       None         4.0           None
 4                                       None         NaN     

In [203]:
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows


def write_tables_to_excel(file_path, sheet_name, tables_list, start_row=7, buffer=3):
    """
    Write a list of DataFrames to an Excel sheet as tables.

    Args:
        file_path (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to write to.
        tables_list (list): List of DataFrames to write.
        start_row (int): Starting row for the first table.
        buffer (int): Number of blank rows between tables.
    """
    from openpyxl import Workbook
    from openpyxl import load_workbook
    from openpyxl.utils import get_column_letter

    # Load the workbook or create a new one
    try:
        wb = load_workbook(file_path)
    except FileNotFoundError:
        wb = Workbook()
        if sheet_name not in wb.sheetnames:
            wb.create_sheet(title=sheet_name)
    ws = wb[sheet_name]

    current_row = start_row
    table_id = 1

    for idx, df in enumerate(tables_list):
        # Determine the range for the table
        start_col = 1  # Assuming writing starts at column A
        end_col = start_col + len(df.columns) - 1
        end_row = current_row + len(df)

        table_range = f"{get_column_letter(start_col)}{current_row}:{get_column_letter(end_col)}{end_row}"
        table_name = f"Table{table_id}"

        # Write the DataFrame to the sheet
        for row in dataframe_to_rows(df, index=False, header=True):
            for col_idx, value in enumerate(row, start=start_col):
                ws.cell(row=current_row, column=col_idx, value=value)
            current_row += 1

        # Add buffer
        current_row += buffer

        # Create an Excel table
        table = Table(displayName=table_name, ref=table_range)
        style = TableStyleInfo(
            name="TableStyleMedium9",  # Choose any predefined Excel style
            showFirstColumn=False,
            showLastColumn=False,
            showRowStripes=True,
            showColumnStripes=False,
        )
        table.tableStyleInfo = style
        ws.add_table(table)
        table_id += 1

    # Save the workbook
    wb.save(file_path)
    print(f"File saved to: {file_path}")



In [186]:
import pandas as pd
import numpy as np

def process_dfs_based_on_identifier(dfs):
    # Define the desired column lists for each keyword
    desired_columns_map = {
        "salaried employees": [
            "Employee Number", "List Order", "Name", "%FTE", 
            "Actual Fringe Rate (104, 131, 136 accts)", "Salary", 
            "Fringe Type", "Start Date (if new)", "End Date (if appl.)"
        ],
        "lump sum": [
            "Employee Number", "Teaching", "Name", "%FTE", 
            "Home Dept", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ],
        "undergraduate": [
            "Employee Number", " ", "Name", "%Appt", 
            " ", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ],
        "graduate": [
            "Employee Number", " ", "Name", "%Appt", 
            " ", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ],
        "other department": [
            "Employee Number", " ", "Name", "%Appt", 
            " ", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ]
    }

    processed_dfs = []

    for df in dfs:
        # Extract the identifier from the first column header
        identifier = df.columns[0]
        raw_identifier = str(identifier).lower()  # Column headers as identifier

        # Match the identifier against the keys in the map
        matched_key = next(
            (key for key in desired_columns_map if key in raw_identifier), 
            None
        )

        if matched_key:
            # Get the corresponding desired column list
            desired_columns = desired_columns_map[matched_key]

            # Remove rows where the 'Name' column is NaN
            if "Name" in df.columns:
                df = df.dropna(subset=["Name"])

            # Ensure all desired columns are present in the DataFrame
            for col in desired_columns:
                if col not in df.columns:
                    df.loc[:, col] = np.nan  # Add missing columns as empty

            # Reorder columns to include the identifier column at the start
            processed_columns = [identifier] + [col for col in desired_columns if col in df.columns]
            df = df[processed_columns]

            # Add the processed DataFrame to the list
            processed_dfs.append(df)
        else:
            print(f"Warning: No match found for identifier '{raw_identifier}'.")

    return processed_dfs


In [195]:
# Initialize an empty list to store DataFrames
dataframes_list = []

# Extract table boundaries from the Excel file
tables = apu.extract_table_boundaries(excel_file)

# Loop through each table, process it, and append to the list
for table in tables:
    # Read the table into a DataFrame
    df = apu.read_excel_table(excel_file, "USER INPUT", table)
    
    # Append the DataFrame to the list
    dataframes_list.append(df)

# Reverse the order of the list
up_df = dataframes_list[::-1]

# Output the reversed list
up_df


[  Salaried Employees  List Order              Name
 0               None         1.0       Sarah Davis
 1               None         2.0       Chris Lacey
 2               None         3.0      Sachin Gupte
 3               None         4.0  Curran Cauldwood
 4               None         5.0         Levi Katz
 5               None         6.0  J. Ashleigh Ross
 6               None         8.0         Sam Russo
 7               None         9.0              None
 8               None        10.0              None
 9               None         NaN              None,
   Lump Sum/No Fringe Employees - Instructors  List Order           Name
 0                                       None         1.0  Jill Jacklitz
 1                                       None         2.0   Hanna Barton
 2                                       None         3.0   Instructor 2
 3                                       None         4.0           None
 4                                       None         NaN     

In [197]:
new_dfs = process_dfs_based_on_identifier(up_df)
pd.options.mode.chained_assignment = None  # Suppress the warning


print("Final validation of new_dfs before writing:")
for i, df in enumerate(new_dfs):
    print(f"Table {i+1}: Type: {type(df)}, Shape: {df.shape if isinstance(df, pd.DataFrame) else 'N/A'}")



Final validation of new_dfs before writing:
Table 1: Type: <class 'pandas.core.frame.DataFrame'>, Shape: (7, 10)
Table 2: Type: <class 'pandas.core.frame.DataFrame'>, Shape: (3, 10)
Table 3: Type: <class 'pandas.core.frame.DataFrame'>, Shape: (2, 10)
Table 4: Type: <class 'pandas.core.frame.DataFrame'>, Shape: (4, 10)
Table 5: Type: <class 'pandas.core.frame.DataFrame'>, Shape: (2, 10)


In [242]:
write_tables_to_excel(
    file_path= r"C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\OUTPUTS\CPP FY2025 Input Data Sheet 2 - ALL INPUT DATA_Feb-21-2025_13-53-06.xlsx",
    sheet_name="USER INPUT",
    tables_list= new_dfs,
    start_row=7,
    buffer=3
)

File saved to: C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\OUTPUTS\CPP FY2025 Input Data Sheet 2 - ALL INPUT DATA_Feb-21-2025_13-53-06.xlsx


In [263]:
# THIS CELL IS THE COMPLETE CODE FOR IPS 1 -> IPS 2 

from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter


import app_util as apu
from openpyxl import Workbook
import re
import openpyxl

#sotring all the headings and its values in a list of tuples
# Nomenclature = {"CONTENT OF HEADING", "ROW NUMBER", "COLUMN NUMBER"}
headings =[
    {"UPDATES",3,1},
    {"FRINGE TABLE",15,1},
    {"TERM LEAVE TABLE",},
    {"CPP EMPLOYEES & SALARY DATA"},
    {"COLAS (Cost of Living Increases)"},
    {"FUNDING STRINGS & ACCOUNTING -- SALARY"},
    {"FUNDING STRINGS & ACCOUNTING -- NON-SALARY"},
]

def generate_input_sheet_2(input_sheet_1_path,destination_path):
    wb = Workbook()
    wb.active
    year = re.search(r'FY(\d{4})', input_sheet_1_path).group(1) if re.search(r'FY(\d{4})', input_sheet_1_path) else None
    final_save_path = os.path.join(destination_path,apu.get_save_name(f"CPP FY{year} Input Data Sheet 2 - ALL INPUT DATA"))
    create_presets(wb,input_sheet_1_path,final_save_path)

    wb.save(final_save_path)
    wb.close()
    print("CLOSING FILE NOW")
    return final_save_path

def create_presets(wb, input_sheet_1_path,input_sheet_2_path):
    # setting tab title
    ws = wb.active
    ws.title = f"USER INPUT"

    # TITLE SETTINGS
    ws.row_dimensions[1].height= 28.50
    title_cell = ws.cell(row=1, column=1)
    title_cell.value = f"INPUT DATA TABLE 2 - ALL INPUT DATA"
    title_cell.font = Font(size=22, bold=True)

    # TIME STAMP SETTINGS
    time_stamp_cell = ws.cell(row=1, column=10)
    apu.apply_color(time_stamp_cell, "maroon")
    time_stamp_cell.value = "Generated on " + datetime.now().strftime("%b-%d-%Y") + " at " + datetime.now().strftime("%H:%M:%S")
    time_stamp_cell.font = Font(size=14, color="FFFFFF", bold=True)  
    ws.merge_cells(start_row=1, start_column=10, end_row=1, end_column=16) 

    # Set width of the first 200 columns and height of the first 200 rows
    for col in range(1, 201):  # Columns A to GR
        ws.column_dimensions[openpyxl.utils.get_column_letter(col)].width = 25
    for row in range(3, 201):  # Rows 1 to 200
        ws.row_dimensions[row].height = 25

    print_employee_and_salary_data(input_sheet_1_path,wb)

    print("PRESETS COMPLETED")

    return

def print_employee_and_salary_data(input_sheet_1_path,wb):
    # Initialize an empty list to store DataFrames
    df_list = []

    # Extract table boundaries from the Excel file
    tables = apu.extract_table_boundaries(input_sheet_1_path)

    # Loop through each table, process it, and append to the list
    for table in tables:
        # Read the table into a DataFrame
        df = apu.read_excel_table(input_sheet_1_path, "USER INPUT", table)
        
        # Append the DataFrame to the list
        df_list.append(df)

    # Reverse the order of the list
    df_list = df_list[::-1]
    df_list = process_dfs_based_on_identifier(up_df)

    write_tables_to_excel(
        wb= wb,
        sheet_name="USER INPUT",
        tables_list=  df_list,
        start_row=7,
        buffer=3
    )

    return



def print_headers(wb):
    
    return  


def process_dfs_based_on_identifier(dfs):

    # Define the desired column lists for each keyword
    desired_columns_map = {
        "salaried employees": [
            "Employee Number", "List Order", "Name", "%FTE", 
            "Actual Fringe Rate (104, 131, 136 accts)", "Salary", 
            "Fringe Type", "Start Date (if new)", "End Date (if appl.)"
        ],
        "lump sum": [
            "Employee Number", "Teaching", "Name", "%FTE", 
            "Home Dept", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ],
        "undergraduate": [
            "Employee Number", " ", "Name", "%Appt", 
            " ", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ],
        "graduate": [
            "Employee Number", " ", "Name", "%Appt", 
            " ", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ],
        "other department": [
            "Employee Number", " ", "Name", "%Appt", 
            " ", "Salary", "Fringe Type", 
            "Start Date (if new)", "End Date (if appl.)"
        ]
    }

    processed_dfs = []

    for df in dfs:
        # Extract the identifier from the first column header
        identifier = df.columns[0]
        raw_identifier = str(identifier).lower()  # Column headers as identifier

        # Match the identifier against the keys in the map
        matched_key = next(
            (key for key in desired_columns_map if key in raw_identifier), 
            None
        )

        if matched_key:
            # Get the corresponding desired column list
            desired_columns = desired_columns_map[matched_key]

            # Remove rows where the 'Name' column is NaN
            if "Name" in df.columns:
                df = df.dropna(subset=["Name"])

            # Ensure all desired columns are present in the DataFrame
            for col in desired_columns:
                if col not in df.columns:
                    df.loc[:, col] = np.nan  # Add missing columns as empty

            # Reorder columns to include the identifier column at the start
            processed_columns = [identifier] + [col for col in desired_columns if col in df.columns]
            df = df[processed_columns]

            # Add the processed DataFrame to the list
            processed_dfs.append(df)
        else:
            print(f"Warning: No match found for identifier '{raw_identifier}'.")

    return processed_dfs



def write_tables_to_excel(wb, sheet_name, tables_list, start_row=7, buffer=3):
    """
    Write a list of DataFrames to an Excel sheet as tables.

    Args:
        wb (Workbook): Excel workbook object.
        sheet_name (str): Name of the sheet to write to.
        tables_list (list): List of DataFrames to write.
        start_row (int): Starting row for the first table.
        buffer (int): Number of blank rows between tables.
    """
    if sheet_name not in wb.sheetnames:
        wb.create_sheet(title=sheet_name)

    ws = wb[sheet_name]
    current_row = start_row

    for idx, df in enumerate(tables_list):
        # Ensure DataFrame is not empty
        if df.empty:
            print(f"Skipping empty DataFrame at index {idx}")
            continue

        # Ensure DataFrame has unique column headers
        if len(df.columns) != len(set(df.columns)):
            print(f"DataFrame at index {idx} has duplicate column headers. Skipping.")
            continue

        # Use the first column header as the base for the table name
        first_header = str(df.columns[0])
        # Sanitize the table name
        sanitized_header = ''.join(e if e.isalnum() else '_' for e in first_header)
        table_name = f"Table_{sanitized_header}_{idx + 1}"

        # Define the range for the table
        start_col = 1  # Writing starts at column A
        end_col = start_col + len(df.columns) - 1
        end_row = current_row + len(df)

        table_range = f"{get_column_letter(start_col)}{current_row}:{get_column_letter(end_col)}{end_row}"

        # Write the DataFrame to the sheet
        for row in dataframe_to_rows(df, index=False, header=True):
            for col_idx, value in enumerate(row, start=start_col):
                ws.cell(row=current_row, column=col_idx, value=value)
            current_row += 1

        # Add buffer
        current_row += buffer

        # Validate the table range before creating the table
        if len(df) > 0 and len(df.columns) > 0:
            try:
                # Create an Excel table
                table = Table(displayName=table_name, ref=table_range)
                style = TableStyleInfo(
                    name="TableStyleMedium9",  # Choose any predefined Excel style
                    showFirstColumn=False,
                    showLastColumn=False,
                    showRowStripes=True,
                    showColumnStripes=False,
                )
                table.tableStyleInfo = style
                ws.add_table(table)
                print(f"Successfully added table: {table_name} with range: {table_range}")
            except ValueError as e:
                print(f"Error creating table {table_name}: {e}")
        else:
            print(f"Skipping DataFrame at index {idx} due to invalid dimensions.")

    print(f"All tables written to sheet '{sheet_name}' successfully.")





In [266]:
import app_util as apu

#TEMPORARILY HARD CODED VALUE FOR TESTING PURPOSES
input = r"C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\CPP FY2025 Input Data Sheet 1 - Employees & Funding Strings.xlsx"
ouput = r"C:\Users\dpaul5\Desktop\2025 APP TESTING\INPUT SHEET TESTING\OUTPUTS"
os.startfile(generate_input_sheet_2(input, ouput))



Successfully added table: Table_Salaried_Employees_1 with range: A7:J14
Successfully added table: Table_Lump_Sum_No_Fringe_Employees___Instructors_2 with range: A18:J21
DataFrame at index 2 has duplicate column headers. Skipping.
DataFrame at index 3 has duplicate column headers. Skipping.
DataFrame at index 4 has duplicate column headers. Skipping.
All tables written to sheet 'USER INPUT' successfully.
PRESETS COMPLETED
CLOSING FILE NOW
